In [ ]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# training data
TRAIN_DATA = [
    ('The C.I.A. director, Gina Haspel, appears to have struck a balance between sticking to her principles while also keeping the C.I.A. mostly free from President Trump’s ire.CreditCreditSarah Silbiger/The New York Times\n', {'entities': [(21, 32, 'CIADIR')]}), 
    ('WASHINGTON — Gina Haspel was trying to brief President Trump early in her tenure as the C.I.A. director, but he appeared distracted. Houseflies buzzing around the Oval Office were drawing his attention, and ire.\n', {'entities': [(13, 24, 'CIADIR')]}), 
    ('An earlier version of this article incorrectly described the photos that Gina Haspel showed to President Trump during a discussion about responding to the nerve agent attack in Britain on a former Russian intelligence officer. Ms. Haspel displayed pictures illustrating the consequences of nerve agent attacks, not images specific to the chemical attack in Britain. This correction was delayed because of the time needed for research.\n', {'entities': [(73, 84, 'CIADIR')]}), 
    ('WASHINGTON (AP) — CIA Director Gina Haspel is headed to Capitol Hill to brief Senate leaders Tuesday on the slaying of Saudi journalist Jamal Khashoggi as senators weigh their next steps in possibly punishing the longtime Middle East ally over the killing.\n', {'entities': [(31, 42, 'CIADIR')]}), 
    ('Trump awaited a briefing Thursday from CIA Director Gina Haspel, who has been in Turkey.\n', {'entities': [(52, 63, 'CIADIR')]}), 
    ('LOUISVILLE, Ky. (AP) — Recalling her rise through male-dominated ranks, CIA Director Gina Haspel said she wants to champion diversity at the spy agency as she returned to her alma mater Monday for a public event where she outlined her priorities for the agency and disclosed a few personal details of her life\n', {'entities': [(85, 96, 'CIADIR')]}), 
    ('— CIA Director Gina Haspel will return to the University of Louisville next week to discuss national security and her career in the spy agency.\n', {'entities': [(15, 26, 'CIADIR')]}), 
    ('WASHINGTON (AP) — Newly declassified documents released Friday graphically describe how an accused al-Qaida operative was stripped naked, repeatedly slammed against walls, waterboarded and confined in boxes for hours at a covert detention site that CIA Director Gina Haspel briefly oversaw after 9/11.\n', {'entities': [(262, 273, 'CIADIR')]}), 
    ('Vaughn Bishop, who has years of experience analyzing intelligence, will serve under CIA Director Gina Haspel, who worked undercover for years on the operations side of the agency.\n', {'entities': [(97, 108, 'CIADIR')]}), 
    ('LANGLEY, Va. (AP) — New CIA Director Gina Haspel pledged to send more spies into the field during her swearing-in Monday as President Donald Trump offered up praise for the rank-and-file, who felt snubbed during his first visit to the headquarters of the premier U.S. intelligence agency.\n', {'entities': [(37, 48, 'CIADIR')]}), 
    ("Newly sworn-in CIA Director Gina Haspel says she wants to send more officers into the field, improve foreign language proficiency among the ranks and strengthen the agency's working relationships with intelligence agencies in partner nations.\n", {'entities': [(28, 39, 'CIADIR')]}), 
    ("McCaskill was one of the few Democrats facing a difficult re-election this fall to oppose the nomination of Gina Haspel, who was confirmed by the Senate on Thursday after a heated debate about her role in the CIA's torture program.\n", {'entities': [(108, 119, 'CIADIR')]}), 
    ('WASHINGTON (AP) — Scrutiny of the 33-year spy career of new CIA director Gina Haspel has focused on her undercover role in the harsh interrogation of suspected terrorists, but she cut her teeth in intelligence operations against Russia.\n', {'entities': [(73, 84, 'CIADIR')]}), 
    ("WASHINGTON (AP) — Veteran spy Gina Haspel will become the first female director of the CIA after six Democrats joined Republicans in a Senate confirmation vote that overrode concerns about her role in the spy agency's harsh interrogation program after 9/11.\n", {'entities': [(30, 41, 'CIADIR')]}), 
    ("CIA Director Gina Haspel's Speech at the University of Louisville\n", {'entities': [(13, 24, 'CIADIR')]}),
    ("On September 24, 2018, CIA Director Gina Haspel returned to her home state of Kentucky to speak at the University of Louisville, as part of the McConnell Center's Distinguished Speaker Series. Director Haspel shared her perspective on leading the Agency and outlined her six strategic priorities.\n", {'entities': [(36, 47, 'CIADIR')]}), (' "Gina Haspel Assumes Role of Acting Director of the Central Intelligence Agency" (Press release). Central Intelligence Agency. April 26, 2018. Retrieved May 2, 2018.\n', {'entities': [(3, 14, 'CIADIR')]}), (' "Gina Haspel Selected to be Deputy Director of CIA". Central Intelligence Agency. February 2, 2017. Archived from the original on February 3, 2017. Retrieved February 2, 2017. Ms. Haspel is the first female career CIA officer to be named Deputy Director.\n', {'entities': [(3, 14, 'CIADIR')]}), (' Glenn Greenwald (February 2, 2017). "The CIA\'s New Deputy Director Ran a Black Site for Torture". The Intercept. Retrieved February 3, 2017. That CIA official\'s name whose torture activities the Post described is Gina Haspel. Today, as BuzzFeed\'s Jason Leopold noted, CIA Director Mike Pompeo announced that Haspel was selected by Trump to be Deputy Director of the CIA.\n', {'entities': [(214, 225, 'CIADIR')]}), (' "Gina Haspel becomes first female CIA deputy director". WDSU. February 2, 2017. Archived from the original on February 2, 2017. Retrieved February 3, 2017.\n', {'entities': [(3, 14, 'CIADIR')]}), (' Harris, Shane (May 17, 2018). "Gina Haspel confirmed as CIA chief despite scrutiny of her role in interrogation program". The Washington Post. Retrieved May 17, 2018 – via MSN. The Senate voted Thursday to confirm Gina Haspel as the next CIA director after several Democrats were persuaded to support her despite lingering concerns about her role in the brutal interrogation of suspected terrorists captured after 9/11.\n', {'entities': [(33, 44, 'CIADIR'), (215, 226, 'CIADIR')]}), (' Berenson, Tessa (March 13, 2018). "Who Is Gina Haspel, President Trump\'s Pick for CIA Director?". Time. Retrieved March 13, 2018.\n', {'entities': [(43, 54, 'CIADIR')]}), (' Suman Varandani (February 3, 2017). "Who Is Gina Haspel? 5 Facts About Trump\'s CIA Deputy Director Pick". International Business Times. Retrieved February 3, 2017. Haspel joined the Central Intelligence Agency in 1985, and spent most of her career undercover. She has been part of several controversies, including her involvement in several torture programs conducted by the U.S. She also ran waterboarding and other interrogation techniques at some of CIA\'s "black sites" or secret prisons. She has not yet been indicted for war crimes.\n', {'entities': [(45, 56, 'CIADIR')]}), (' Holpuch, Amanda (March 13, 2018). "Who is Gina Haspel? Trump\'s pick for CIA chief linked to torture site". The Guardian.\n', {'entities': [(43, 54, 'CIADIR')]}), (' Allen, Nick (March 13, 2018). "Gina Haspel: Donald Trump\'s new CIA director ran torture site in Thailand". The Daily Telegraph.\n', {'entities': [(33, 44, 'CIADIR')]}), (' Washington, Boer Deng (March 14, 2018). "New CIA chief Gina Haspel helped run torture site in Thailand". The Times.\n', {'entities': [(56, 67, 'CIADIR')]}), (' Ward, Alex (March 13, 2018). "Gina Haspel, Trump\'s CIA director pick, oversaw the torture of dozens of people". Vox.\n', {'entities': [(32, 43, 'CIADIR')]}), (' Mora, Alberto (March 15, 2018). "Gina Haspel Is a Torturer. What Else Does the Senate Need to Know?". Politico Magazine. Retrieved March 16, 2018.\n', {'entities': [(35, 46, 'CIADIR')]}), (' "CIA chief Gina Haspel faces a grilling". The Australian. March 18, 2018. Even the most basic facts about Ms Haspel\'s life are hard to establish. She was born Gina Cherie [sic] Walker in Kentucky in 1956. At 20, she married Jeff Haspel, an army officer, but they were divorced by the time she joined the CIA in 1985 as a reports officer, specializing in Russia. By 1988, she was listed as "acting head of administration" at the US embassy in Addis Ababa. ... Her subsequent postings remain classified but she was based in Ankara in 2003 and was CIA station chief in New York.\n', {'entities': [(12, 23, 'CIADIR')]}), (' Youssef, Nancy A. (March 22, 2018). "CIA Fills In Some Blanks on Gina Haspel\'s Secret Life". The Wall Street Journal. Archived from the original on March 22, 2018. Retrieved March 22, 2018. She became a spy before the internet age and remained in that secret life for three decades, leaving behind no digital profile. ... So it falls to the agency to share something about her and her interests.\n', {'entities': [(66, 77, 'CIADIR')]}), (' Myre, Greg (April 20, 2018). "The CIA Introduces Gina Haspel After Her Long Career Undercover". NPR. Retrieved April 27, 2018.\n', {'entities': [(50, 61, 'CIADIR')]}), (' "CIA chief Gina Haspel faces a grilling". The Australian. March 18, 2018. Even the most basic facts about Ms Haspel\'s life are hard to establish. She was born Gina Cherie [sic] Walker in Kentucky in 1956. At 20, she married Jeff Haspel, an army officer, but they were divorced by the time she joined the CIA in 1985 as a reports officer, specialising in Russia. By 1988, she was listed as "acting head of administration" at the US embassy in Addis Ababa. ... Her subsequent postings remain classified but she was based in Ankara in 2003 and was CIA station chief in New York.\n', {'entities': [(12, 23, 'CIADIR')]}), (' Gazis, Olivia (May 1, 2018). "CIA Director nominee Gina Haspel: CIA releases timeline of her clandestine career". CBS News. Retrieved May 1, 2018.\n', {'entities': [(52, 63, 'CIADIR')]}), (' "Gina Haspel". Central Intelligence Agency. Archived from the original on March 14, 2018. Retrieved March 13, 2018.\n', {'entities': [(3, 14, 'CIADIR')]}), (' Goldman, Adam (March 13, 2018). "Gina Haspel, Trump\'s Choice for C.I.A., Played Role in Torture Program". The New York Times. Retrieved March 14, 2018.\n', {'entities': [(35, 46, 'CIADIR')]}), (' Carol Rosenberg (January 8, 2019). "Did CIA Director Gina Haspel run a black site at Guantánamo?". McClatchy News Service. Guantanamo. Archived from the original on January 8, 2019. The claim by Rita Radostitz, a lawyer for Khalid Sheik Mohammed, appears in one paragraph of a partially redacted transcript of a secret hearing held at Guantánamo on Nov. 16. Defense lawyers were arguing, in a motion that ultimately failed, that Haspel’s role at the prison precludes the possibility of a fair trial for the men accused of orchestrating the 9/11 attacks who were also held for years in covert CIA prisons.\n', {'entities': [(54, 65, 'CIADIR')]}), (' "Redacted-transcript-of-closed-9-11-trial-hearing". Guantanamo Military Commission. November 16, 2018. Archived from the original on January 8, 2019. Retrieved January 8, 2019. And so again, our evidence here is that there is a change, a significant change, a sea change in the classification guidance once Gina Haspel becomes in a position of power within the CIA. And we don \'t know for sure, and we cannot tell you for sure that she is who requested that change in the classification guidance.  Media related to File:Redacted-transcript-of-closed-9-11-trial-hearing_(2018-11-16).pdf at Wikimedia Commons\n', {'entities': [(308, 319, 'CIADIR')]}), (' Rosenberg, Matthew (February 2, 2017). "Gina Haspel, C.I.A. Deputy Director, Had Role in Torture". The New York Times. Retrieved March 17, 2018.\n', {'entities': [(42, 53, 'CIADIR')]}), (' Barnes, Julian; Shane, Scott (August 10, 2018). "Cables Detail C.I.A. Waterboarding at Secret Prison Run by Gina Haspel". The New York Times. Retrieved August 10, 2018.\n', {'entities': [(109, 120, 'CIADIR')]}), (' Parton, Heather Digby (March 17, 2018). "New Report on CIA Nominee Gina Haspel May Rescue Her – But It Shouldn\'t". Salon.com. Retrieved March 25, 2018.\n', {'entities': [(68, 79, 'CIADIR')]}), (' Prados, John (May 9, 2018). "The CIA Black Sites Program and the Gina Haspel Nomination". The National Security Archive.\n', {'entities': [(66, 77, 'CIADIR')]}), (' Miller, Greg; Harris, Shane (March 13, 2018). "Gina Haspel, Trump\'s pick for CIA director, tied to use of brutal interrogation measures". The Washington Post.\n', {'entities': [(49, 60, 'CIADIR')]}), (' Savage, Charlie (May 8, 2018). "9/11 Planner, Tortured by C.I.A., Asks to Tell Senators About Gina Haspel". The New York Times. Retrieved May 8, 2018.\n', {'entities': [(95, 106, 'CIADIR')]}), (' Nunes, Devin (February 2, 2017). "Nunes Statement on Appointment of Gina Haspel as CIA Deputy Director" (Press release). US House of Representatives Permanent Select Committee on Intelligence. Archived from the original on March 14, 2017. Retrieved March 13, 2018.\n', {'entities': [(69, 80, 'CIADIR')]}), (' Katie Bo Williams (February 8, 2017). "Third Dem urges removal of Trump\'s pick for top CIA deputy". The Hill. Archived from the original on February 10, 2017. Retrieved February 14, 2017. Trump\'s pick of 30-year veteran Gina Haspel to serve as deputy director of the CIA – which is not a Senate-confirmable position – has reinvigorated fears that the administration is weighing a return to the use of banned techniques now considered torture, such as waterboarding and sleep deprivation.\n', {'entities': [(221, 232, 'CIADIR')]}), (' Spencer Ackerman (February 15, 2017). "Deputy CIA director could face court deposition over post-9/11 role in torture". The Guardian. Retrieved February 15, 2017. In a court filing on Tuesday, attorneys for two CIA contract psychologists who helped design the agency\'s brutal interrogations for terrorism suspects have asked a federal judge to order Gina Haspel, a career CIA officer recently appointed as the agency\'s No2 official, to provide a deposition discussing her allegedly pivotal involvement in an episode the CIA has tried repeatedly to put behind it.\n', {'entities': [(351, 362, 'CIADIR')]}), (' Press, Associated (March 13, 2018). "Gina Haspel named by Trump to be CIA director, replacing Pompeo, who will replace Tillerson at State". The Washington Post. Associated Press. Retrieved March 13, 2018.\n', {'entities': [(39, 50, 'CIADIR')]}), (' @realdonaldtrump (March 13, 2018). "Mike Pompeo, Director of the CIA, will become our new Secretary of State. He will do a fantastic job! Thank you to Rex Tillerson for his service! Gina Haspel will become the new Director of the CIA, and the first woman so chosen. Congratulations to all!" (Tweet). Archived from the original on March 13, 2018 – via Twitter.\n', {'entities': [(183, 194, 'CIADIR')]}), (' Baer, Robert (March 19, 2018). "Gina Haspel Used To Work For Me. She\'s A Great Choice For CIA Director". NPR. Retrieved March 20, 2018.\n', {'entities': [(34, 45, 'CIADIR')]}), (' Smith, David (March 14, 2018). "Rand Paul to oppose Gina Haspel as CIA director over her \'gleeful joy\' at torture". The Guardian. Retrieved March 15, 2018.\n', {'entities': [(53, 64, 'CIADIR')]}), (' He, Alan (March 16, 2018). "CIA Director nominee Gina Haspel did not mock tortured detainee". CBS News. Retrieved March 17, 2018.\n', {'entities': [(50, 61, 'CIADIR')]}), (' Frank, Thomas (March 23, 2018). "Fatima Boudchar Was Bound, Gagged And Photographed Naked. John McCain Wants To Know If Gina Haspel\'s Okay With That". Buzzfeed News. Retrieved April 5, 2018.\n', {'entities': [(121, 132, 'CIADIR')]}), (' Demirjian, Karoun (April 25, 2018). "CIA refuses to declassify more information about Gina Haspel, Trump\'s pick to lead the agency". The Washington Post. Retrieved April 27, 2018.\n', {'entities': [(87, 98, 'CIADIR')]}), (' Greenwald, Glenn (May 8, 2018). "Will Democrats Unite to Block Trump\'s Torturer, Gina Haspel, as CIA Chief? If Not, What Do They #Resist?". The Intercept. Retrieved May 15, 2018.\n', {'entities': [(82, 93, 'CIADIR')]}), (' Editorial (May 10, 2018). "Gina Haspel fails the test". The Washington Post. Retrieved May 15, 2018.\n', {'entities': [(29, 40, 'CIADIR')]}), (' Pengelly, Martin (May 12, 2018). "Gina Haspel: Democrat Donnelly backs CIA pick beset by torture questions". The Guardian. Retrieved May 15, 2018.\n', {'entities': [(36, 47, 'CIADIR')]}), (' Myre, Greg (May 15, 2018). "Gina Haspel: CIA Should Not Have Carried Out \'Enhanced Interrogation\'". NPR. Retrieved May 17, 2018.\n', {'entities': [(30, 41, 'CIADIR')]}), (' Fandos, Nicholas (May 17, 2018). "Senate Confirms Gina Haspel to Lead C.I.A. Despite Torture Concerns". The New York Times. Retrieved May 19, 2018.\n', {'entities': [(51, 62, 'CIADIR')]}), (' Herb, Jeremy; Diaz, Daniella (May 17, 2018). "Controversial nominee Gina Haspel confirmed as first female CIA director". CNN. Retrieved May 19, 2018.\n', {'entities': [(69, 80, 'CIADIR')]}), (' Strobel, Warren P. (May 25, 2019). "Under CIA Chief Gina Haspel, an Intelligence Service Returns to the Shadows". The Wall Street Journal. Retrieved May 28, 2019. The CIA\'s first female director since its 1947 founding, she has put in place her own leadership team—which also includes many women—and so far has avoided having President Trump\'s political allies embedded in the agency\'s senior ranks.\n', {'entities': [(53, 64, 'CIADIR')]}), (' Jansen, Bart (March 13, 2018). "Gina Haspel, nominated by Trump as first woman to lead CIA, has controversial past". USA Today. Retrieved March 13, 2018.\n', {'entities': [(34, 45, 'CIADIR')]}), (' Manson, Katrina (March 16, 2018). "Gina Haspel, the undercover spy picked to head the CIA". Financial Times. Retrieved March 22, 2018.\n', {'entities': [(37, 48, 'CIADIR')]})]

    


# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int),
#                 )

In [ ]:
def main(model="en_core_web_sm", output_dir="r", n_iter=20):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.load("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is  None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
        for text, _ in TRAIN_DATA:
            doc = nlp(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
            # save model to output directory
        if output_dir is not None:
            output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
            nlp.to_disk(output_dir)
            print("Saved model to", output_dir)
                # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])
        


In [ ]:
main()


Loaded model 'en_core_web_sm'
Losses {'ner': 3247.705643057823}
Losses {'ner': 3409.1126967668533}
Losses {'ner': 3241.6285878419876}
Losses {'ner': 3206.926466345787}
Losses {'ner': 3243.9153954982758}
Losses {'ner': 3094.1094057559967}
Losses {'ner': 3118.3688005954027}
Losses {'ner': 3087.6721307884436}
Losses {'ner': 3033.0278737545013}
Losses {'ner': 2988.0686274170876}
Losses {'ner': 3026.7400188446045}
Losses {'ner': 2891.564383506775}
Losses {'ner': 2927.129473090172}
Losses {'ner': 2879.8219990730286}
Losses {'ner': 2916.8974299430847}
Losses {'ner': 2876.873636484146}
Losses {'ner': 2843.3601217865944}
Losses {'ner': 2821.5839297175407}
Losses {'ner': 2827.072573661804}
Losses {'ner': 2836.957826461643}
Entities []
Tokens [('An', '', 2), ('earlier', '', 2), ('version', '', 2), ('of', '', 2), ('this', '', 2), ('article', '', 2), ('incorrectly', '', 2), ('described', '', 2), ('the', '', 2), ('photos', '', 2), ('that', '', 2), ('Gina', '', 2), ('Haspel', '', 2), ('showed', '', 2

In [ ]:
nlp3=spacy.load("./r")

In [ ]:
nlp4 = spacy.load("en_core_web_sm")

In [ ]:
doc=nlp3("Jack Cheri Haspel is an American intelligence officer serving as Director of the Central Intelligence Agency since 2018. She is the first woman to hold the post on a permanent basis and was previously the Deputy Director under Mike Pompeo during the early presidency of Donald Trump.")
for x in doc.ents:
  print(x.text,x.label_)

an American intelligence CIADIR


In [ ]:
if __name__ == "__main__":
    plac.call(main)